## Setup Dependencies

In [1]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

### AWS CLI and AWS Python SDK (boto3)

In [2]:
!pip install --disable-pip-version-check -q awscli boto3

### SageMaker

In [3]:
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments

### PyAthena

In [4]:
!pip install --disable-pip-version-check -q PyAthena

### AWS Data Wrangler

In [5]:
!pip install --disable-pip-version-check -q awswrangler

### Zip

In [6]:
!conda install -y zip

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



### Matplotlib

In [7]:
!pip install --disable-pip-version-check -q matplotlib

### Seaborn

In [8]:
!pip install --disable-pip-version-check -q seaborn

## Data lake Setup

In [9]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = 'flightfinalapproachanomalydetection'#'sagemaker-us-east-1-817854913925' #sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

print("Bucket name: {}".format(bucket))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Bucket name: flightfinalapproachanomalydetection


### Verify Bucket Creation

In [10]:
from botocore.client import ClientError

response = None
try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e: 
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'M6HB5XKYFC2JBQYM', 'HostId': 'sTxybdIuY5xgekwQ/4nIp4R9LpZf8fkcPedNr1rkaHKLsjraTHZv1UQFaZjERNb8LZi+V+k0qw4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'sTxybdIuY5xgekwQ/4nIp4R9LpZf8fkcPedNr1rkaHKLsjraTHZv1UQFaZjERNb8LZi+V+k0qw4=', 'x-amz-request-id': 'M6HB5XKYFC2JBQYM', 'date': 'Tue, 21 Jan 2025 03:37:35 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [11]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name 
account_id = boto3.client(service_name="sagemaker", region_name=region)

In [12]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/flightfinalapproachanomalydetection/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_8957/88463448.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


## Create Athena Database Scheme
### Import PyAthena

In [43]:
from pyathena import connect

In [44]:
table_name_csv = "flight_csv_data"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

### Create Athena Database

In [45]:
database_name = "flightdata_db"

In [46]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [47]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [50]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS flightdata_db


In [51]:
cursor = conn.cursor()
cursor.execute(statement)
print("Database created successfully")


Database created successfully


In [52]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_8957/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,assignment2_aws
1,default
2,dsoaws
3,flightdata_db


## Register CSV with Athena

In [53]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS flightdata_db.flight_csv_data (
        sample_id int,
        timestep int,
        aileron_pos_lh_deg float,
        aileron_pos_rh_deg float,
        corrected_angle_of_attack_deg float,
        baro_correct_alt_lsp_ft float,
        computed_airspeed_lsp_knots float,
        selected_course_deg float,
        drift_angle_deg float,
        elevator_pos_left_deg float,
        te_flap_pos_disc float,
        glideslope_dev_perc float,
        selected_heading_deg float,
        localizer_dev_perc float,
        core_speed_avg_perc float,
        total_pressure_lsp_millibar float,
        pitch_angle_lsp_deg float,
        roll_angle_lsp_deg float,
        rudder_pos_deg float,
        true_heading_lsp_deg float,
        vertical_accel_g float,
        wind_speed_knots float      
)
STORED AS TEXTFILE
LOCATION 's3://flightfinalapproachanomalydetection/csv/'
TBLPROPERTIES (
    'skip.header.line.count'='1',
    'field.delim'=',',
    'compressionType'='none'
)"""

In [54]:
cursor = conn.cursor()
cursor.execute(statement)

In [55]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_8957/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [56]:
statement = "SHOW TABLES IN {}".format(database_name)
cursor = conn.cursor()
cursor.execute(statement)

In [57]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_csv_passed = True
else:
    ingest_create_athena_table_csv_passed = False
print(ingest_create_athena_table_csv_passed)

False


In [59]:
print(database_name)
print(table_name_csv)

flightdata_db
flight_csv_data


In [65]:
statement = """SELECT sample_id, aileron_pos_lh_deg, timestep FROM {}.{}
    LIMIT 10""".format(
    database_name, table_name_csv
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT sample_id, aileron_pos_lh_deg, timestep FROM flightdata_db.flight_csv_data
    LIMIT 10


/tmp/ipykernel_8957/1251818097.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,sample_id,aileron_pos_lh_deg,timestep
0,6606,86.27341,119
1,6606,86.29386,120
2,6606,91.10149,121
3,6606,84.12531,122
4,6606,86.76440,123
